# Note Classification

In [ ]:
## Declare Bucket Paths

In [1]:
from google.cloud import storage
bucket_name = "team4-project"

storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
my_prefix='imgs/euro/'
blobs = bucket.list_blobs(prefix = my_prefix, delimiter = '/')
print(blobs)
for blob in blobs:
    print(blob)

<Blob: team4-project, imgs/euro/.DS_Store, 1650537665680586>
<Blob: team4-project, imgs/euro/full_list.csv, 1650641657886222>
<Blob: team4-project, imgs/euro/train_list.csv, 1650641639609143>
<Blob: team4-project, imgs/euro/train_set.csv, 1650622880765520>
<Blob: team4-project, imgs/euro/val_list.csv, 1650641641574812>


In [ ]:
# Get a list of all files in teh path

In [ ]:
blobs_all = list(bucket.list_blobs(prefix=my_prefix))
data_out=[]
for x in blobs_all:
    #Select only image files
    if x.name.endswith(('.png','.jpg','.jpeg')):
        data_out.append(['gs://'+bucket_name+'/'+x.name,x.name.split('/')[-2]])

 

                                                     0    1
0     gs://team4-project/imgs/euro/10/10-Euro-back.jpg   10
1          gs://team4-project/imgs/euro/10/10-Euro.jpg   10
2    gs://team4-project/imgs/euro/10/10euro_fr_ES1_...   10
3    gs://team4-project/imgs/euro/10/10euro_re_ES1_...   10
4    gs://team4-project/imgs/euro/10/ECB_10euro_Ban...   10
..                                                 ...  ...
254  gs://team4-project/imgs/euro/500/Screen Shot 2...  500
255  gs://team4-project/imgs/euro/500/Screen Shot 2...  500
256  gs://team4-project/imgs/euro/500/Screen Shot 2...  500
257  gs://team4-project/imgs/euro/500/Screen Shot 2...  500
258  gs://team4-project/imgs/euro/500/Screen Shot 2...  500

[259 rows x 2 columns]
Copying file://text.csv [Content-Type=text/csv]...
/ [1 files][ 16.8 KiB/ 16.8 KiB]                                                
Operation completed over 1 objects/16.8 KiB.                                     


In [ ]:
## Export full csv list

In [ ]:
import pandas as pd

df = pd.DataFrame(data_out)
print(df    )     
df.to_csv('text.csv', index = False,header=False)
full_out_path=f'gs://{bucket_name}/{my_prefix}full_list.csv'
!gsutil cp 'text.csv' $full_out_path


## Export train and validatino samples

In [3]:
train_data = df.sample(frac = 0.8)
train_data.to_csv('train_data.csv', index = False,header=False)
train_out_path=f'gs://{bucket_name}/{my_prefix}train_list.csv'
!gsutil cp 'train_data.csv' $train_out_path

# Creating dataframe with
# rest of the 50% values
val_data = df.drop(train_data.index)
val_data.to_csv('val_data.csv', index = False,header=False)
val_out_path=f'gs://{bucket_name}/{my_prefix}val_list.csv'
!gsutil cp 'val_data.csv' $val_out_path


Copying file://train_data.csv [Content-Type=text/csv]...
/ [1 files][ 13.4 KiB/ 13.4 KiB]                                                
Operation completed over 1 objects/13.4 KiB.                                     
Copying file://val_data.csv [Content-Type=text/csv]...
/ [1 files][  3.4 KiB/  3.4 KiB]                                                
Operation completed over 1 objects/3.4 KiB.                                      


In [4]:
train_data_loc=train_out_path
val_data_loc = val_out_path
CLASS_NAMES = df[1].unique()
print(train_data_loc)
print(val_data_loc)
print("These are the available classes:", CLASS_NAMES)

gs://team4-project/imgs/euro/train_list.csv
gs://team4-project/imgs/euro/val_list.csv
These are the available classes: ['10' '100' '20' '200' '5' '50' '500']


In [5]:
train_data_loc="gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/train_set.csv"
val_data_loc="gs://practical-ml-vision-book/flowers_5_jpeg/flower_photos/eval_set.csv"
CLASS_NAMES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
print("These are the available classes:", CLASS_NAMES)

These are the available classes: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [6]:
import tensorflow as tf
print(tf.version.VERSION)
#GPU doesn't work!
'''
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
'''

2.8.0


"\ndevice_name = tf.test.gpu_device_name()\nif device_name != '/device:GPU:0':\n    raise SystemError('GPU device not found')\nprint('Found GPU at: {}'.format(device_name))\n"

In [7]:
# Helper functions
def training_plot(metrics, history):
  f, ax = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 5))
  for idx, metric in enumerate(metrics):
    ax[idx].plot(history.history[metric], ls='dashed')
    ax[idx].set_xlabel("Epochs")
    ax[idx].set_ylabel(metric)
    ax[idx].plot(history.history['val_' + metric]);
    ax[idx].legend([metric, 'val_' + metric])

# Call model.predict() on a few images in the evaluation dataset
def plot_predictions(model, filename):
  f, ax = plt.subplots(7, 5, figsize=(25,15))
  dataset = (tf.data.TextLineDataset(filename).
      map(decode_csv))
  for idx, (img, label) in enumerate(dataset.take(35)):
    ax[idx//5, idx%5].imshow((img.numpy()));
    batch_image = tf.reshape(img, [1, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS])
    batch_pred = model.predict(batch_image)
    pred = batch_pred[0]
    label = CLASS_NAMES[label.numpy()]
    pred_label_index = tf.math.argmax(pred).numpy()
    pred_label = CLASS_NAMES[pred_label_index]
    prob = pred[pred_label_index]
    ax[idx//5, idx%5].set_title('{}: {} ({:.4f})'.format(label, pred_label, prob))
    ax[idx//5, idx%5].axis('off')

def show_trained_weights(model):
  # CLASS_NAMES is ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
  LAYER = 1 # Layer 0 flattens the image, layer=1 is the first dense layer
  WEIGHT_TYPE = 0 # 0 for weight, 1 for bias

  f, ax = plt.subplots(1, 5, figsize=(15,15))
  for flower in range(len(CLASS_NAMES)):
    weights = model.layers[LAYER].get_weights()[WEIGHT_TYPE][:, flower]
    min_wt = tf.math.reduce_min(weights).numpy()
    max_wt = tf.math.reduce_max(weights).numpy()
    flower_name = CLASS_NAMES[flower]
    print("Scaling weights for {} in {} to {}".format(
        flower_name, min_wt, max_wt))
    weights = (weights - min_wt)/(max_wt - min_wt)
    ax[flower].imshow(weights.reshape(IMG_HEIGHT, IMG_WIDTH, 3));
    ax[flower].set_title(flower_name);
    ax[flower].axis('off')
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_CHANNELS = 3

def read_and_decode(filename, reshape_dims):
  # Read the file
  img = tf.io.read_file(filename)
  # Convert the compressed string to a 3D uint8 tensor.
  img = tf.image.decode_jpeg(img, channels=IMG_CHANNELS)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # Resize the image to the desired size.
  return tf.image.resize(img, reshape_dims)



# the label is the index into CLASS_NAMES array
def decode_csv(csv_row):
  record_defaults = ["path", "flower"]
  filename, label_string = tf.io.decode_csv(csv_row, record_defaults)
  img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])
  label = tf.argmax(tf.math.equal(CLASS_NAMES, label_string))
  return img, label

In [8]:
import tensorflow_hub as hub
import os
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'

# parameterize to the values in the previous cell
def train_and_evaluate(batch_size = 32,
                       lrate = 0.001,
                       l1 = 0.,
                       l2 = 0.,
                       num_hidden = 16):
  regularizer = tf.keras.regularizers.l1_l2(l1, l2)

  train_dataset = (tf.data.TextLineDataset(
      train_data_loc).
      map(decode_csv)).batch(batch_size)

  eval_dataset = (tf.data.TextLineDataset(
      val_data_loc).
      map(decode_csv)).batch(32) # this doesn't matter

  layers = [
      hub.KerasLayer(
          "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
          input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS),
          trainable=False,
          name='mobilenet_embedding'),
      tf.keras.layers.Dense(num_hidden,
                            kernel_regularizer=regularizer, 
                            activation='relu',
                            name='dense_hidden'),
      tf.keras.layers.Dense(len(CLASS_NAMES), 
                            kernel_regularizer=regularizer,
                            activation='softmax',
                            name='flower_prob')
  ]

  model = tf.keras.Sequential(layers, name='flower_classification')
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lrate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(
                    from_logits=False),
                metrics=['accuracy'])
  print(model.summary())
  history = model.fit(train_dataset, validation_data=eval_dataset, epochs=5)
  training_plot(['loss', 'accuracy'], history)
  return model

In [ ]:
model = train_and_evaluate()


2022-04-22 16:02:23.305012: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "flower_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_embedding (KerasL  (None, 1280)             2257984   
 ayer)                                                           
                                                                 
 dense_hidden (Dense)        (None, 16)                20496     
                                                                 
 flower_prob (Dense)         (None, 5)                 85        
                                                                 
Total params: 2,278,565
Trainable params: 20,581
Non-trainable params: 2,257,984
_________________________________________________________________
None
Epoch 1/5
104/104 [==============================] - 155s 1s/step - loss: 0.6406 - accuracy: 0.7718 - val_loss: 0.4086 - val_accuracy: 0.8622
Epoch 2/5
 37/104 [=========>....................] - ETA: 1:24 - loss: 0.3036 - a

In [ ]:
print(val_data_loc)
plot_predictions(model, val_data_loc)